In [2]:
import os

In [3]:
%pwd

'd:\\Chicken_Disease_ENd_to_End\\research'

In [4]:
os.chdir('../')

In [5]:
%pwd

'd:\\Chicken_Disease_ENd_to_End'

In [6]:
from dataclasses import dataclass
from pathlib import Path


In [7]:
@dataclass(frozen=True)
class CallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path
    
    

In [8]:
from ChickenDisease.constants import *
from ChickenDisease.utils.common import read_yaml, create_directories

In [9]:
#read_yaml(CONFIG_FILE_PATH).prepare_callbacks
model_ckpt_dir = os.path.dirname(read_yaml(CONFIG_FILE_PATH).prepare_callbacks.checkpoint_model_filepath)
model_ckpt_dir

[2023-07-28 00:04:00,036: INFO: common: yaml file: config\config.yaml loaded successfully]


'artifacts/prepare_callbacks/checkpoint_dir'

In [10]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_callback_config(self) -> CallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        
        create_directories([Path(model_ckpt_dir), Path(config.tensorboard_root_log_dir)])
        
        prepare_callback_config = CallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )
        
        return prepare_callback_config
        

In [11]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [12]:
# config:CallbacksConfig
# config = config
#print(config.tensorboard_root_log_dir)

In [13]:
class PrepareCallback:
    def __init__(self, config:CallbacksConfig):
        self.config = config
       # print(config)
    @property
    def create_tb_callback(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(self.config.tensorboard_root_log_dir,
                                          f"tb_logs_at_{timestamp}",)
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def ckpt_callback(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
        
    
    def get_tb_ckpt_callbacks(self):
        return [
            self.create_tb_callback,
            self.ckpt_callback
        ]

In [14]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_callback_config()
    prepare_callbacks = PrepareCallback(config = prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
except Exception as e:
    raise e

[2023-07-28 00:04:10,073: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-28 00:04:10,084: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-28 00:04:10,087: INFO: common: created directory at: artifacts]
[2023-07-28 00:04:10,090: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2023-07-28 00:04:10,094: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
